Cross-language Information Retreival

```Student Name: Yifan Yu
Student ID: 702550```

### Warning: File encodings and tokenisation

All the text files are encoded in *utf-8*, which requires some care in using with python strings, the nltk tools and jupyter.  Please use the following method to convert strings into ASCII by escaping the special symbols. Be careful to do the conversion after tokenisation, as the escaped tags might interfere with the NLTK tokenizer and get treated as punctuation.

In [ ]:

# from nltk.tokenize import word_tokenize

# def tokenize(line, tokenizer=word_tokenize):
#     utf_line = line.decode('utf-8').lower()
#     return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

# tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

# Part 1: CLIR engine

The project has two parts: first you must build a CLIR engine, comprising information retrieval and translation components, and evaluate its accuracy. Next you will propose an extension to a component of the system. In each step you will need to justify your modelling and implementation decisions, and evaluate the quality of the outputs of the system, and at the end you will need to reflect on the overall outcomes.

The CLIR system will involve:
 - translating queries from German into English, the language of our text collection, using word-based translation
 - once the queries and the documents are in the same language, search over the document collection using BM25
 - evaluate the quality of ranked retreival results using the query relevance judgements

Note that you could try translating the text collection into German instead, but for this project we'll stick with translating the query.

Building the CLIR engine is the majority of work in the assignment, and  constitutes 70% of the assignment mark. Note that although there are several steps, they do not necessarily need to be attempted in a linear order. That is, some components are independent of others. If you're struggling with one component and cannot complete it, then you may be able to skip over it and return to it later. Where outputs are needed in a subsequent step you may want to consider ways of side-stepping this need, e.g., by using Google translate output rather than your system output for translating the queries. You should aim to answer all components.

In [34]:
import os
def computeMap(filename):
    eva={}
    cidset=set()
    for line in open(relative_path(filename)):            
        cid, rank, sales = line.rstrip('\n').split(",")
        cidset.add(cid)
        eva[(cid,rank)]=sales
    l=list(cidset)
    l.sort(key = int)
    total=0.0
    for cid in l:
        i=1
        count=0.0
        precision=0.0
        while i<k and ((cid,str(i)) in eva.keys()):
            if eva[(cid,str(i))]=='1':
                count+=1
                precision+=count/i
            i+=1
        if count!=0:
            p_ave=precision/count
            total+=p_ave
    map=total/len(cidset)
    return map

def relative_path(filename):
    base_path = os.path.dirname(os.path.abspath("__file__"))
    file_path = os.path.join( base_path +'\\'+ filename)
    return file_path

k=5
start =time.clock()
print "map= ",computeMap('Evaluation.csv')
end =time.clock()
print "Run time:",(end-start)


map=  0.175557324841
Run time: 0.813018136762


NameError: name 'eva' is not defined

## Information Retreival with BM25

You should implement a vector-space retreival system using the BM25 approach. You'll need to:
 - load in the data files and tokenize the input,
 - preprocess the lexicon, e.g., by stemming and stop-word removal, 
 - calculate the TF/IDF representation for all documents in the collection,
 - store an inverted index such that you can efficiently recover the documents matching a query term, and
 - implement querying in the BM25 model
 - test that it runs with some English queries (you'll have to make these up)
 
This should run in a reasonable time over *dev.docs* (up to a few minutes to index), but beyond this does not need to be highly optimised. Feel free to use python dictionaries, sets, lists, numpy/scipy matrices etc where appropriate for best runtime, but you shouldn't need to use specialised data structures such as compression schemes or the like. Feel free to use APIs in NLTK, scikit-learn and other allowed python modules as needed (see list above.) You will probably want to save and load your index to/from disk to save repeated re-indexing every time you load the file.

In [ ]:
#predefine and import the usual functions
import nltk
import os
import re
import time
import string

stopwords = set(nltk.corpus.stopwords.words('english')) 
stemmer = nltk.stem.PorterStemmer()
punctuation = set(string.punctuation)
punctuation.add('``')

# use the function given for decode every inputs
def decode(line):
    utf_line = line.decode('utf-8').lower()
    line =  utf_line.encode('ascii', 'backslashreplace') 
    return line

# find the relative path of a file
def relative_path(filename):
    base_path = os.path.dirname(os.path.abspath("__file__"))
    file_path = os.path.join( base_path + '\\data_files_ass2\\'+ filename)
    return file_path

#Common preprocess technique including decode, tokenize and stemming 
def preprocess_line(line):
    words=[]
    line=decode_encode(line)
    for token in tokenizer(line):
        words.append(stemmer.stem(token))
    return words

In [ ]:
# load files and preprocess lexicon

def preprocess(filename):
    docs={}
    doc_list={}
    doc_id_list=[]
    lengdoc={}
    aveLeng=0.0
    totallength=0.0
    dir= relative_path(filename)
    f=open(dir)
    for line in f:
        doc_id,text=line.split("\t")
        text = decode(text)
#       terms as the set of words for inverted index
#       tokens as the ones stored in doc for processing
        tokens,terms = lexicon(text)
        docs[doc_id]= terms
        onedoc=" ".join(tokens)
        doc_list[doc_id]=onedoc
        doc_id_list.append(doc_id)
        lengdoc[doc_id]=len(doc_list[doc_id])
        totallength+=lengdoc[doc_id]
    numberofdocs=len(lengdoc)
    aveLeng=totallength/float(numberofdocs)
#   precompute all these value for better performance in query BM25
    return docs,lengdoc,aveLeng,numberofdocs,doc_list,doc_id_list

def lexicon(doc):
    words=[]
    terms = set()
    for token in doc.split():
        if token not in stopwords: 
            term=stemmer.stem(token)
            words.append(term)
            terms.add(term)
    return words,terms

start =time.clock()

docs,lengdoc,aveLeng,n,doc_list,doc_id_list=preprocess('dev.docs')
# print docs
# print lengdoc
# print n
# print aveLeng
end=time.clock()

print "Run time:",(end-start)

In [ ]:
#inverted index,create the inverted index for easy access to the doc contain certain term
from collections import defaultdict

def getInvertedIndex(docs):
    inverted_index = defaultdict(list)
    for docid, terms in docs.items():
        for term in terms:
            inverted_index[term].append(docid)
    # need to keep doc lists in sorted order
    for term, index in inverted_index.items():
        index.sort()

    return inverted_index


start =time.clock()

inverted_index = getInvertedIndex(docs)

end=time.clock()

print "Run time:",(end-start)

# print inverted_index

In [ ]:
#implement querying in the BM25 model
# test that it runs with some English queries

#dummy query for testing 

query="anarch polit philosophi plural modern"

#the parameter setting in BM25 model
k1=1.5
k3=0
b=0.5

def calculate_bm25(n,ft,lendoc,aveLeng,fdt,fqt,k1,k3,b):
    import math
    idf=math.log((n-ft+0.5)/(ft + 0.5))
    tf=float((k1 + 1) * fdt) /(k1*((1-b)+b*lendoc/aveLeng)+fdt)
    querytf=((k3+1) *fqt) / (k3 + fqt)
    score= idf * tf * querytf
    return score

# get the bm25 for a certain doc according to the query given
def queryDoc(query,docs,wordsinQuery,lengdoc,invented_index):
    score=0
    for word in wordsinQuery:
        fqt =get_fqt(word,query)
        fdt =get_fdt(word,docs)
        doc_ids=set()
        for ids in inverted_index[word]:
            doc_ids.add(ids)
        ft = len(doc_ids)
        score += calculate_bm25(n,ft,lengdoc,aveLeng,fdt,fqt,k1,k3,b)
    return score

def get_fqt(word,query):
    fqt= query.split().count(word)
    return fqt

def get_fdt(word,doc):
    fdt= doc.split().count(word)
    return fdt

# do one query for all the documents given and output the best 10 results with highest BM25
def query_One (query,doc_list,inverted_index):
    rank = {}
    wordsinQuery=set()
    wordsinQuery=query.split()
    docContainQuery_id=set()
    for words in wordsinQuery:
        for docids in inverted_index[words]:
            docContainQuery_id.add(docids)
#   print docContainQuery_id
    for id in docContainQuery_id:
        result = queryDoc(query,doc_list[id],wordsinQuery,lengdoc[id],inverted_index)
        rank[id] = result
#     print rank_result_dict
    result = sorted(rank.items(), key=lambda d:d[1], reverse=True)
#     print result
    return result[:10]

print "Query: "+query
print "The most relevent docs according to query: "
print query_One (query,doc_list,inverted_index)

In computing BM25 score, firstly, I tokenize the query into tokens, and use inverted index find the documents contain words in the query, because the documents that do not contain query have no score. Then, use the precomputed value and query information to calculate the BM25 score for all the documents. Then, Sorted the result dict and output in a descending order.

## Translating the queries 

For translation, you should implement a simple word-based translation model in a noisy channel setting, where you search for the best translation string using a language model over the English with a translation model over the back-translation of the English output string, $\vec{e}$, into the German input string, $\vec{f}$. This corresponds to finding the string, $\vec{e}$ which maximises $p(\vec{e}) p(\vec{f} | \vec{e})$. This has two components:

### Language model

The first step is to estimate a language model. You should learn both a unigram language model and a trigram language model, and compare the two. Note that the unigram will be used in the *decoding* step below. You will have to think about how to smooth your estimates, and the related problem of handling unknown words.

Please use *bitext-large.en* to train your models (start with *bitext-small.en* for your development) and evaluate the perplexity of your model on *newstest2013.en*. You should justify your development decisions and parameter settings in a markdown cell, and quantify their effect on perplexity, including the differences you notice between the unigram and the trigram models.

In [ ]:
from collections import defaultdict
from nltk.tokenize import word_tokenize as tokenizer
import os
import math

filename_bismall = "bitext-small.en"
filename_bilarge = "bitext-large.en"
filename_test = "newstest2013.en"

def load(filename):
    docs={}
    doc_list=[]
    docid=0
    dir= relative_path(filename)
    f=open(dir,'r')
    lines = f.readlines()
    for line in lines:
        words=[]
        terms = decode(line)
        for term in tokenizer(terms):
            words.append(term)
        doc_list.append(words)
    return doc_list

def check_for_unk_train(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        unigram_counts[word] = 0
        return "UNK"

#     put the start and end symbol to every single sentence
def convert_sentence_train(sentence,unigram_counts):
    return ["<s1>"] + ["<s2>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s2>"] + ["</s1>"]

#    get the trigram, bigram, unigram counts by the single method 
def get_counts(sentences):
    trigram_counts = defaultdict(lambda: defaultdict(dict))
    bigram_counts = defaultdict(dict)
    unigram_counts = {}
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 2):
            trigram_counts[sentence[i]][sentence[i+1]][sentence[i+2]] = trigram_counts[sentence[i]][sentence[i+1]].get(sentence[i+2],0)+1
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    token_count = float(sum(unigram_counts.values()))
#   as the loop stops before unigram count the end symbal, make it the same probability count with the start.
    unigram_counts["</s1>"] = unigram_counts["<s1>"]
    unigram_counts["</s2>"] = unigram_counts["<s2>"]
    return unigram_counts, bigram_counts, trigram_counts, token_count

#   use the log function for computing the trigram probability by using interpolation as smoothing strategy.
def get_log_prob_interp(sentence,i, unigram_counts,bigram_counts,trigram_counts,token_count, trigram_lambda):
    return math.log(trigram_lambda*trigram_counts[sentence[i-2]][sentence[i-1]].get(sentence[i],0)/float(bigram_counts[sentence[i-2]].get(sentence[i-1],1)) + \
                    (1 - trigram_lambda)* trigram_lambda * bigram_counts[sentence[i-1]].get(sentence[i],0)/float(unigram_counts[sentence[i-1]]) + \
                    (1 - trigram_lambda)*(1-trigram_lambda) * unigram_counts[sentence[i]]/token_count)

def check_for_unk_test(word,unigram_counts):
    if word in unigram_counts and unigram_counts[word] > 0:
        return word
    else:
        return "UNK"

def convert_sentence_test(sentence,unigram_counts):
    return ["<s1>"] + ["<s2>"]+ [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s2>"] + ["</s1>"]

def get_sent_log_prob_interp(sentence, unigram_counts, bigram_counts,trigram_counts, token_count, trigram_lambda):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_interp(sentence,i, unigram_counts,bigram_counts,trigram_counts,token_count,trigram_lambda) for i in range(2,len(sentence))])

# create the training set and test set randomly if only one dataset given for held out testing.
def separte_sets(docs):
    from random import shuffle
    sents = docs
    shuffle(sents)
    cutoff = int(0.8*len(sents))
    training_set = sents[:cutoff]
    test_set = [[word.lower() for word in sent] for sent in sents[cutoff:]]
    return training_set,test_set

training_set=[]
test_set=[]

# doc_list=load(filename_bismall)
# doc_list=load(filename_bilarge)
# (training_set,test_set)=separte_sets(doc_list)

training_set=load(filename_bilarge)
test_set=load(filename_test)

unigrams, bigrams,trigrams, token_count = get_counts(training_set)

# calculate the perplexity of the model for evaluation.
def calculate_perplexity(sentences,unigram_counts,bigram_counts,trigram_counts, token_count, smoothing_function, parameter):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 1 # have to consider the end token
        total_log_prob += smoothing_function(sentence,unigram_counts,bigram_counts,trigram_counts,token_count, parameter)
    return math.exp(-total_log_prob/test_token_count)

# several lambdas are tested in order to get the optimum value
for trigram_lambda in [0.99,0.95,0.75,0.5,0.4,0.3,0.25,0.1,0.001]:
    print trigram_lambda
    print calculate_perplexity(test_set,unigrams,bigrams,trigrams,token_count,get_sent_log_prob_interp,trigram_lambda)

print "unigrams perplexity"
print calculate_perplexity(test_set,unigrams,bigrams,trigrams,token_count,get_sent_log_prob_interp,0)
# print "token_counts:"
# print token_count

The unigram model and trigram model are trained in bitext_large.en as training set and tested on the newstest2013.en
The result above shows that the trigram model performs better than unigram model in most cases.
During several experiments, the best fit point of lambda in interpolation is found as 0.4, which is a low compatively. Somehow, it could be given a general idea that for higher -grams model, the parameter lambda may lose its meaning when it's too high or too low. Several other experiments were done as in the following slot:
The unigram model and trigram model also been tested on held-out data with a 80% / 20% seperation in bitext-small and bitext-large. Some pattern are shown that unigram model perform better in the small set and trigram model perform better in the large set. It could be believed that with much larger training set, the optimum lambda for trigram would be higher and the perplexity would be smaller generally.

In [ ]:
Trigram 
training set:bi-large.en
test set:newstest2013.en

0.99
4448.41176954
0.95
1418.72440082
0.75
486.973221341
0.5
354.25373319
0.4
345.080939219
0.3
353.685571458
0.25
366.24679724
0.1
473.61623981
0.001
963.792904825
unigrams perplexity
1025.36245872

Trigram bi-small 80/20 held out

0.99
5732.59178758
0.95
1402.43859919
0.75
371.883510992
0.5
241.929565207
0.4
228.342449343
0.3
227.323122287
0.25
232.024607199
0.1
285.419292972
0.001
537.378609594
unigrams perplexity
574.191464949

Trigram bi-large 80/20 held out
0.99
1789.02446246
0.95
668.919718607
0.75
273.013977735
0.5
218.545393563
0.4
220.135843356
0.3
233.784980403
0.25
246.934505363
0.1
347.199256535
0.001
904.893302967
unigrams perplexity
1017.76382647


### Translation model

The next step is to estimate translation model probabilities. For this we will use a word-based alignment model, e.g., *ibm1* to learn word-based translation probabilities using expectation maxisation. Your task is to obtain good quality word alignments from this data, which will require careful use of the word alignment models. You will want to training in both translation directions, and combining the alignments.

You should use the *bitext-small* files for this purpose, and be aware that it may take a minute or two to train ibm1 on this data. Inspect some of the word alignments and translation probabilities (e.g., using a few common words in German such as *haus*) to see if your approach is working, and give some examples of output alignments that you find that are good and bad. 

In [ ]:
from nltk.translate import IBMModel1
from nltk.translate import AlignedSent, Alignment
import os
import operator
from nltk.tokenize import word_tokenize as tokenizer
import time
import nltk
import string

stopwords_ger = set(nltk.corpus.stopwords.words('german')) 
stopwords_eng = set(nltk.corpus.stopwords.words('english'))

punctuation = set(string.punctuation)
filename_Eng = "bitext-small.en"
filename_Ger = "bitext-small.de"

# load the two text file and preprocess them  for further use
def load_file(fileE,fileF):
    text_e=[]
    text_f=[]
    dirE= relative_path(fileE)
    dirF= relative_path(fileF)
    f1=open(dirE,'r')
    f2=open(dirF,'r')
    lines1 = f1.readlines()
    lines2 = f2.readlines()
    for line in lines1:
        line=decode(line)
#         line = ' '.join([word for word in tokenizer(line) if word not in stopwords_eng and word not in punctuation])
        line = ' '.join([word for word in tokenizer(line) if word not in punctuation])
#         for token in tokenizer(line):
#             if token not in punctuation and word not in stopwords_eng:
#                 line = ' '.join()
        text_e.append(line)
#         line= removestopwords(line,'eng')
    for line in lines2:
        line=decode(line)
#         line = ' '.join([word for word in tokenizer(line) if word not in stopwords_ger and word not in punctuation])
        line = ' '.join([word for word in tokenizer(line) if word not in punctuation])
        text_f.append(line)
#         line= removestopwords(line,'ger')        
    return text_e,text_f

# create the bitext of combining the english and german docs in a list
def get_bitext(list1,list2):
    bitext = []
    for i in range(len(list1)):
        bitext.append((list1[i].split(),list2[i].split()))
        #bitext.append(((list1[i].split()),(list2[i].split())))
    return bitext

start =time.clock()

text_e,text_f=load_file(filename_Eng, filename_Ger)

bitext_EngtoGer = get_bitext(text_e,text_f)
bitext_GertoEng = get_bitext(text_f,text_e)

end = time.clock()

print('Running time: %s Seconds'%(end-start))
# bitext_EngtoGer = get_bitext(text_e[:10],text_f[:10])
# # bitext_GertoEng = get_bitext(text_f[:5],text_e[:5])

In [ ]:
# train the translation model IBM1 from German to English and form English to German using the bitext
# IBM1 model was trained for 10 iterations in order to get a better result

start =time.clock()

bt_EngtoGer=[AlignedSent(E,F) for E,F in bitext_EngtoGer]
bt_GertoEng=[AlignedSent(E,F) for E,F in bitext_GertoEng]

m_EngtoGer=IBMModel1(bt_EngtoGer, 10)
m_GertoEng=IBMModel1(bt_GertoEng, 10)

end = time.clock()

print('Running time: %s Seconds'%(end-start))

# m_EngtoGer.translation_table
# bitext = []
# bitext.append(("green house".split(), "casa verde".split()))
# bitext.append(("the house".split(), "la casa".split()))

# print bitext


# bt = [AlignedSent(E,F) for E,F in bitext]
# m = IBMModel1(bt, 10)

# m.translation_table
# max(m.translation_table['green'].iteritems(), key=operator.itemgetter(1))[0]

In [ ]:
print max(m_GertoEng.translation_table['haus'].iteritems(), key=operator.itemgetter(1))[0]
# when it come to the noun word like 'haus' which is 'house', it is easy to get the right result. 

print max(m_EngtoGer.translation_table['carbon'].iteritems(), key=operator.itemgetter(1))[0]
# the "carbon" is a rare seen noun but it also translate into the right meaning as "kohlenstoff", which could generate a idea that the direct 
# translation model of ibm1 performs pretty well in translating nouns

print max(m_EngtoGer.translation_table['power'].iteritems(), key=operator.itemgetter(1))[0]
# The first two translation are very good, but the translation from 'power' to 'macht' is debateble as the meaning of 'power' in english veries
# a lot, "macht" means the strength and power of right, which is a part of the power mean. However, when it's talking about electricity power,the
# translation is bad. It's a problem for most of the word based translation algorithm that not able to adjust to the context.

print max(m_GertoEng.translation_table['der'].iteritems(), key=operator.itemgetter(1))[0]
print max(m_EngtoGer.translation_table['and'].iteritems(), key=operator.itemgetter(1))[0]
print max(m_EngtoGer.translation_table['the'].iteritems(), key=operator.itemgetter(1))[0]
# these are two typical bad translations, the first one is pretty wired as it should be "the" in English but it translated into "detering"
# which doesn't make any sense at all, it could be the problem that some of the english words are embedded in the German corpus which means the 
# training set is now well enough that some of the english words are tend not to translate at all in these two cases


In [ ]:
# combining the alignments 

#  print bitext_EngtoGer[0]
# print bt_EngtoGer[0].alignment[0][0][0]
# print bt_EngtoGer[0].alignment[0][0]
# print bt_EngtoGer[0].alignment[0]
# print bt_EngtoGer[0].alignment
# print bt_GertoEng[0].alignment
# print bt_EngtoGer[0]

def union(bt_EngtoGer,bt_GertoEng):
    union=list()
    for i in range(len(bt_EngtoGer)):
        alignments=list()
        for align1 in bt_EngtoGer[i].alignment:
            alignments.append(align1)
        for align2 in bt_GertoEng[i].alignment:
            if align2 not in alignments:
                alignments.append(align2)
        union.append(alignments)
    return union

def inter(bt_EngtoGer,bt_GertoEng):
    inter=list()
    for i in range(len(bt_EngtoGer)):
        alignments=list()
        alignments_inter=set()
        for align1 in bt_EngtoGer[i].alignment:
            alignments.append(align1)
        for align2 in bt_GertoEng[i].alignment:
            if align2 in alignments:
                alignments_inter.add(align2)
            else:
                for align3 in alignments:
                    if ((align2[0]==align3[1]) and (align3[0]==align2[1])):
                        alignments_inter.add(align3)
        inter.append(alignments_inter)
    return inter

inter_list=inter(bt_EngtoGer,bt_GertoEng)
union_list=union(bt_EngtoGer,bt_GertoEng)
    
for i in range(3):
    print "Original bi_text:"
    print  bitext_EngtoGer[i]
    print "intersection:"
    print inter_list[i]
    print "union:"
    print union_list[i]
    print '\n'

The first one and the third one is pretty short which is not a good one for analyzing the alignments.
Take the second sentence as a example.The intersection shows that 'gold' align to 'gold', of align to 'von' and ... until 'conversation' align to 'gespr\\xe4ch' which is a consecutive 6 words alignment, which strongly suggest that it's a possible phrase which is exactly a right phase alignment according to their meaning. Also the unions near the intersection are potential words could make a phrase alaignment.

### Decoding 

Finally you'll need to solve for the best translation for a given string. For this you should do simple word-for-word translation where each word is translated independently. Use the alignments learned above (or the translation parameters) to translate each word of the queries into English. Compare taking the maximum probability of translation into English $p(e|f)$, with the noisy-channel probability, $p(f|e)p(e)$, which considers the reverse translation and a unigram language model probability. (You'll get a chance later to do something more ambitious, using the trigram model.)

Use your algorithms to translate the first 100 queries into English, and save them to disk.

In [ ]:
# load the query and preprocess
import string
import nltk
import re

stopwords_ger = set(nltk.corpus.stopwords.words('german')) 
punctuation = set(string.punctuation)
punctuation.add('``')
filename='dev.queries'


def preprocess(filename):
    doc_list={}
    dir= relative_path(filename)
    f=open(dir,'r')
#     the first 100 queries
    lines = f.readlines(100)
    num = 0
    for line in lines:
        num+=1
        doc_id,text = line.split("\t")
#       only int could be sorted in a right docs order
        doc_id=int(doc_id)
        if num ==100:
            break
        text = decode(text)
        terms = list()
        for word in tokenizer(text):
#           romove the punctuation and stopwords which could show the meaningful words in the query 
            if word not in punctuation and word not in stopwords_ger:
                terms.append(word)
        doc_list[doc_id] = terms
    return doc_list

query_list=preprocess(filename)
output_list = sorted(query_list.items(), key=lambda d:d[0])
print output_list

In [ ]:
# taking the maximum probability of translation into English p(e|f)

stopwords_eng = set(nltk.corpus.stopwords.words('english'))

def find_largest_prob(word):
    return max(m_GertoEng.translation_table[word].iteritems(), key=operator.itemgetter(1))[0]

def translate_one_query(sentence):
    eng_sentence=""
    for word in sentence:
#         print m_GertoEng.translation_table[word]
        if word in m_GertoEng.translation_table:
            eng_word=str(find_largest_prob(word))
            if eng_word=="None":
                eng_word=""
        else:
            eng_word= ""
        if eng_word=="":
            eng_sentence=eng_sentence
        else:
            eng_sentence=eng_sentence+eng_word+" "
    return eng_sentence

def translate_query_list(query_dic):
    result_dic={}
    for key in query_list:
        result_dic[key]=translate_one_query(query_list[key])
    output_list = sorted(result_dic.items(), key=lambda d:d[0])
    for key in output_list:
        print "Queryid: "+str(key[0])
        print "Translation Result: "+key[1]    
    return output_list

start =time.clock()

result_maxprob=translate_query_list(output_list)
# write the first 100 queries into the disc
filepath = "data_files_ass2/translation_maxprob.txt"
f1 = open(filepath,"w")
for i in range(len(result_maxprob)):
    f1.write(str(result_maxprob[i][0])+'\t'+result_maxprob[i][1]+'\n')
f1.close()


end = time.clock()

print('Running time: %s Seconds'%(end-start))
    
# translate_one_query(query_list)


In [ ]:
# the noisy-channel probability, p(f|e)p(e)
# p(e):
#   Unigram model of the bi_text_large.en is used to calculate the laguagemodel score
# p(f|e):
#   Ibm1 model trained before From English to German used in calculating the Translation model score 
import operator

unigrams, bigrams,trigrams, token_count = get_counts(training_set)
w_lm=1
w_tl=1

# language model which rewards a better english word
def unigramScore(query,unigrams,token_count):
    if query in stopwords_eng:
#       down scaling the stopwords as the stopwords have much higher unigram probablility
        score= float(unigrams[query])/(float(token_count)*10000)
    else:
        score= float(unigrams[query])/float(token_count)
    return score

# try to find the key in translation_table that form a dict of Eng words could possiblly translated to query
def find_key(query,translation_table):
    potential_dic={}
    for key,values in translation_table.items():
        if query in values:
            if translation_table[key][query]>0.001 :
                potential_dic[key]=translation_table[key][query]
    return potential_dic

# find the best matched english words with highest results combining language model and translation model
def best_match(query,translation_table,unigrams,token_count):
    results = {}
    result=""
    potential_dic = find_key(query,translation_table)
#     print result_dic
    if potential_dic =={}:
        return ""
    else:
        for word,value in potential_dic.items():
            lm_score = unigramScore(word,unigrams,token_count)
            tl_score = value
            score = (lm_score)*w_lm*(tl_score)*w_tl
            results[word] = score
    result= max(results.iteritems(), key=operator.itemgetter(1))[0]
    return result

# find the best match english words in the sentence one by one
def translate_sentence(sentence):
    eng_sentence=""
    for word in sentence:
        eng_word=best_match(word,m_EngtoGer.translation_table,unigrams,token_count)
        if eng_word=="":
            continue
        else:
            eng_sentence=eng_sentence+eng_word+" "
    return eng_sentence

def translate_query(query_list):
    result_dic={}
    for key in query_list:
        result_dic[key]=translate_sentence(query_list[key])
    output_list = sorted(result_dic.items(), key=lambda d:d[0])
    for key in output_list:
        print "Queryid: "+str(key[0])
#     t=re.sub("UNK",'',t)
        print "Translation Result: "+key[1]
    return output_list
    
start =time.clock()

# print best_match('haus',m_EngtoGer.translation_table,unigrams,token_count)
result_noisechannel=translate_query(query_list)

filepath = "data_files_ass2/translation_unigram_noisychannel.txt"
f2 = open(filepath,"w")
for i in range(len(result_noisechannel)):
    f2.write(str(result_noisechannel[i][0])+'\t'+result_noisechannel[i][1]+'\n')
f2.close()
end = time.clock()

print('Running time: %s Seconds'%(end-start))

Now you will need to write some text about the procedure above, and how well it worked. You should answer:
  - how good are the translations? How do your methods fare at translating the content words versus functions words? You can use Google translate to provide an alternative (pretty good) translation for comparison purposes. 
  - what are good settings for various modelling parameters such as language model discounting, translation model smoothing, any decoding parameters, and how do these affect the outputs?
  - compare the language model perplexity for the unigram and trigram models over your decoder outputs, how does the difference in perplexity between the two models compare to your test above on the monolingual data? 
  - how do the learned alignments differ between the two translation directions, and does their combination appear oto improve the predictions? (You don't need a formal evaluation here, as we have no *gold standard* alignments.)

--Most of the result of translation is pretty bad, as the context information are all missing no matter for max probability or noise channel method. It could be seen that most of the content word could be well translated comparing the function words. Some of the stopwords like "der" is translated into "detering" which doesn't make sense at all.

--In particular the decoding parameter is pretty important in this case, as the corpu is pretty big that most of the unigram model result in very small number but the stopwords which highly shown in the corpus have a very high prob. In order to have a reasonable result a higher parameter given to the LM and a penalty given to the stopwords lead to a better result.

-- The model with lower perplexity seems more reliable than the small one, especially for the case of comparing trigram modle and unigram model.

--The alignment on both side may give a better understanding because the alignment of the words shows some different characters in these two language. The model gradually learned these patterns which could benefit the improvement of the prediction.

## Putting the CLIR system together 

Now you should couple the translation and IR components from above. Take your translated queries and use these with the IR system to find the most relevant documents. You should then evaluate these predictions against the supplied relevance judgements, using the *mean average precision* (MAP) metric. As part of this, you will want to consider tuning the settings of the IR system for best performance, and comment on how successful your approach was and evaluate the IR performance is affected by the parameter settings and modelling decisions you have made.

Note that if you were stuck on the translation steps above, or your translations are otherwise unusable, you can implement this step using google translate outputs for the 100 queries.

In [ ]:
# output_list could be chosed from the translation results from the max translation 
# probability or max unigram noisy-channel probability

def preprocess_query(doc):
    terms = list()
    doc= decode(doc)
    for token in doc.split():
        if token not in punctuation and token not in stopwords_eng: 
            terms.append(stemmer.stem(token))
    text=" ".join(terms)
    return text

def get_query(query_list):
    q_list={}
    k=0
    for query in query_list:
        if k<10:
            text=preprocess_query(query[1])
            q_list[query[0]]=text
            k+=1
    return q_list

query=get_query(result_maxprob)
# query=get_query(result_noisechannel)

query

In [ ]:
def get_actual_list(filename):
    path= relative_path(filename)
    dic={}
    docs=[]
    f = open(path,'r')
    for line in f:
        docs.append(line.split('\t')[2])
        dic[line.split('\t')[0]]=docs
    return dic


def get_map_score(queryId,resultList,relDic,k):
    resultList.reverse()
    results=resultList[0:k] #[(docid,score),...]
    i = 1
    precision = 0.0
    count=0.0
    while i < k:
        if results[i][0] in relDic[queryId]:
            count+=1
            precision += count/i
        i += 1
    p_ave=precision/count
    return p_ave

k=10
filename= "dev.qrel"
map_dict={}
total_map=0
predicted_list={}

actual_list = get_actual_list(filename)

for queryid,text in query.items():
    print queryid
    predicted_list[queryid]= query_One(text,doc_list,inverted_index)
    print predicted_list[queryid]
    total_map+=get_map_score(queryid,predicted_list[queryid],actual_list[queryid],k)
print total_map/float(len(query))



The mean average precision is used to evaluate the result of translation model within the entire docs and produce the most relevant 100 results for calculation. The result is pretty bad as the unigram noise channel doesn't perform well and either the max probablity model.It could be the problem of the word alignment is not that good in this instance.

Note that in a complete CLIR you would translate the retreived documents back into German. We won't bother about that for this assignment, especially as I doubt many of you can understand German.

# Part 2: Extension and discussion

## Extension

You now have a working end-to-end CLIR system. The next step is to revisit some of the steps above to see if you can improve the performance of the system. You are invited to be creative with your ideas about how to do this. Note that many great ideas might not produce improvements, or end up being very difficult to implement. This will not be assessed solely on the grounds of CLIR accuracy; we will be looking primarily for interesting and creative ideas.

Here are some ideas on what you might do. Warning: some are bigger than others, aim to spend 5-6 hours in total on this.

#### Structured decoding

Decode using a higher order language model, possibly with word reordering. Using your trigram language model and a word-based or phrase-based translation model, search for the best scoring translation sentence. You may want to use the tools in *nltk.translate*, including the phrase extraction and *StackDecoder*, especially if you wish to support word reordering. The decoding algorithm is much simpler (and tractable) without reordering, so you might want to implement this yourself. You will want to work with log-probabilities to avoid numerical underflow. Note  that the *StackDecoder* is not highly stable code, so you will need to understand its code in detail if you chose to use it. You will need to supply translation log probabilities and language model log probabilities as input to the decoder and consider the context of the previous two words in estimating the probability of each word. 

#### Probabilisitc querying

Consider queries including weighted terms based on multiple translation outputs, such as the word translation probabilities or using *k-best* translations from a decoder (you will need to extract not just the best translation from the decode, but the runners up, which can be done from the stack contents after beam search.)  You should consider how best to define TF and DF for query words based on their ambiguous translations. You may find inspiration in the bibliography of [Douglas Oard](https://terpconnect.umd.edu/~oard/research.html#mlia) such as [this paper](https://terpconnect.umd.edu/~oard/pdf/coling12ture.pdf).

#### Word alignment

Implement an extended word alignment model, such as the *hmm* model (see [Vogel's paper](http://www.aclweb.org/anthology/C96-2141)) or a variant of *ibm2*. The variant of *ibm2* could include a better formulation for the distortion term, $p(i|j,I,J)$, based around rewarding values of i and j that are at similar relative positions to encourage alignments near the diagonal. This contrasts with *ibm2* which just learns a massive table of counts for all combinations of i,j,I and J. This strictly needs normalisation, as its not a probability, but for simplicity you can ignore this aspect here. This idea is inspired by [fast-align](https://github.com/clab/fast_align) which has a similar, but slightly more complex, formulation.

You might also want to experiment with [fast-align](https://github.com/clab/fast_align) (an extremely fast and accurate variant of *ibm2*) or [GIZA++](http://www.statmt.org/moses/giza/GIZA++.html) (an implementation of *ibm1* - *ibm5* and the *hmm*), both widely used and highly optimised alignment tools. Neither of these tools are in python, and may be a little involved to install and compile.

#### Decoding

Experiment with a state-of-the-art translation tool like [Moses](http://www.statmt.org/moses/).  Can you get better CLIR performance using this, trained on the full parallel dataset? You might want to consider translating the document collection into German, and then doing the IR entirely in German. You might consider building a python interface to moses, building on the existing very basic funcationality. **Warning:** Moses is a complex suite of software, and can be a little complex to install and use. However there are good installation and usage guides on the moses webpage.

#### IR engine

There are several great open source IR engines, including [Lucene](https://lucene.apache.org/core/), [Terrier](http://terrier.org/), [Zettair](http://www.seg.rmit.edu.au/zettair/) and [Lemur](http://www.lemurproject.org/). You could experiment with these, and use this to compare the accuracy of different types of IR models on the CLIR data. Alternatively, you could implement a faster or more compact retreival system in python, e.g., using compressed vbyte encodings, skip lists or similar.

Note that the development of some of the extensions maye require working in the command shell and in other languages than python. If this is the case, you should attach your code in separate files as part of your submission, but include the analysis text here. *Note that excellent submissions implementing interfaces or models not currently in NLTK will be considered for inclusion into the toolkit.* You should include both the code, and text explaining your work and findings.

In [ ]:
from collections import defaultdict, Counter
import itertools
from nltk.tokenize import word_tokenize
from nltk.translate import *
from math import log

start =time.clock()

bitext_EngtoGer = get_bitext(text_e,text_f)

bt_EngtoGer=[AlignedSent(E,F) for E,F in bitext_EngtoGer]

m=IBMModel1(bt_EngtoGer, 10)

end=time.clock()
print('Running time: %s Seconds'%(end-start))

In [ ]:
# remove the "None" in alignment
align=[]
for i in range(len(bt_EngtoGer)):
    a=bt_EngtoGer[i].alignment
    align.append(str(a))
    if None in align:
        del align[None]


In [ ]:
from nltk.translate.api import PhraseTableEntry
class PhraseTable(object):
    def __init__(self):
        self.src_phrases = dict()
        
    def translations_for(self, src_phrase):
        if len(src_phrase)==1:
            if src_phrase not in self.src_phrases or self.src_phrases[src_phrase] == [] :
                return [PhraseTableEntry(trg_phrase=src_phrase, log_prob=0.0)]
        return self.src_phrases[src_phrase]
    
    def add(self, src_phrase, trg_phrase, log_prob):
        entry = PhraseTableEntry(trg_phrase=trg_phrase, log_prob=log_prob)
        if src_phrase not in self.src_phrases:
            self.src_phrases[src_phrase] = []
        self.src_phrases[src_phrase].append(entry)
        self.src_phrases[src_phrase].sort(key=lambda e: e.log_prob,
                                          reverse=True)
    def __contains__(self, src_phrase):
        if len(src_phrase)==1:
            return True
        return src_phrase in self.src_phrases

In [ ]:
# trigrams LM model
language_prob=get_log_prob_interp(sentence,i, unigram_counts,bigram_counts,trigram_counts,token_count, trigram_lambda)

In [ ]:
# phrase extraction
from nltk.translate.phrase_based import phrase_extraction

phrase_pair_counts = defaultdict(Counter)
for s in bt_EngtoGer:
    if None not in s.alignment:
        phrases = phrase_extraction(' '.join(s.words), ' '.join(s.mots), s.alignment, max_phrase_length=3)
        for en_span, fr_span, en_phrase, fr_phrase in phrases:
            phrase_pair_counts[tuple(en_phrase.split())][tuple(fr_phrase.split())] += 1

phrase_table = PhraseTable()
for en_phrase, fr_counts in phrase_pair_counts.items():
    total = sum(fr_counts.values())
    for fr_phrase, count in fr_counts.items():
        phrase_table.add(fr_phrase, en_phrase, log(count / float(total)))
        
class LM:
    def __init__(self, probs):
        self.probs = probs        
    def probability_change(self, context, phrase): 
        # Used when expanding a hypothesis with a new phrase
        # (higher order LMs would need to look at the word sequence, including context)
        return sum([self.probs[word] for word in phrase])
    def probability(self, phrase): 
        # Used for future cost estimation only, to get a cheap (approximate) LM score for each phrase 
        return sum([self.probs[word] for word in phrase])
language_model = LM(language_prob)

Try to use the stack decoder to build a noise channel model with phrase alignments and trigrams language model for better performance, but stuck in the phrase table forming, the the result of alignments in ibm model contains "None" which is not able to deal with in the phrase. Even I tried to change the code in nltk PhraseTable, still could not deal with "None". Tried to delete the "None" in alignment as suggested in the discussion, but the alaignment type not allowed for del[] function

## Discussion 

What conclusions can you make about CLIR, or more generally translation and IR? Overall what approaches worked and what ones didn't? What insights do you have from doing this, put another way, if you were to start again, what approach would you take and why?

About the CLIR, I think the combination of the translation and information retrival could generate the better. It is a good method to using noise channel algorithm which incorperate the idea of both translation result and useful information from language model.In a lot of circumstance, only using the probability of translation model results into a meaningless translation as the highest probability is rigid. It could be easily find out that in most of the documents, the phrase based alignment could have far more better performance than word alignment. It is pretty obvious as the phrase alignment has better information in the order of the words which is very crucial in this point. 

Additionaly, the translation model like IBM 3 invovle reordering of the resutls alignment which is a much better way to get useful result. But it seems the IBM 3 model trainging process is really time-consuming or it requires more powerful computer to train it because I tried the training of the model and based on the bitext_small given, it couldn't generate the result alignments in 30mins.

Overall the idea of precomputing the data in doc before querying is very helpful, at the start, I didn't store any value in when process the documents, it ended up really slow that take 3 mins to compute one query. After storing everythin i can in the preprocess step, it becomes much quicker then before.